In [168]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json statsmodels
! pip install --quiet backoff xxhash pyyaml geopandas dtreeviz facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [169]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
from environs import Env
from adopt.malaria import get_db_conf

env = Env()
env.read_env('.env-bug', override=True)

db_conf = get_db_conf(env)

In [171]:
import geopandas as gpd
import pandas as pd
from scipy.special import expit


def get_dist_info(path):
    geod = gpd.read_file(path)
    district_info = geod.groupby('disthash').apply(lambda df: df.iloc[0][['disthash', 'tot_p']]).reset_index(drop=True)
    return district_info

dist_stats = get_dist_info('outs/cluster.shp')
treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
dist_info = pd.read_csv('outs/dist_info.csv')

In [214]:
from mnm.prep import *

forms, form_keys = get_form_data(['malaria no more'], db_conf)
rdf = get_response_data(forms, db_conf)

In [218]:
rdf = add_form_metadata(forms, form_keys, rdf)
rdf = add_response_metadata(["clusterid"], rdf)

In [219]:
rdf.shape

(1211754, 12)

In [223]:
testers = rdf[rdf["clusterid"].isna()].userid.unique()
rdf = rdf[~rdf.userid.isin(testers)].reset_index(drop=True)

In [251]:
from mnm.prep import _add_time_indicators

rdf = _add_time_indicators(rdf)

In [252]:
from mnm.prep import get_response_df_round_a

dda = get_response_df_round_a(['malaria no more'], 'http://localhost:1323', db_conf)
dda = dda \
    .merge(treatment_assignment, how='right', left_on='clusterid', right_on='disthash') \
    .merge(dist_info, on='clusterid')

ERROR for destination baselineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


ERROR for destination mnm_endlineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


ERROR for destination baselineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


ERROR for destination mnm_endlineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


ERROR for destination baselineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


ERROR for destination baselineeng: Could not create translation mapping. Failed with the following error: Forms have different lengths!


In [253]:
# NOTE:
# can't do this anymore, need raw df, same for costs.
# dd[dd.wave == 0].to_csv('outs/individual-for-balance.csv', index=False)

# Only pick users who gave answers from September
our_users = set(dda[dda.week > 36].userid.unique())
our_users &= set(dda[dda.wave >= 1].userid.unique())
dda = dda[dda.userid.isin(our_users)]

In [172]:
ddb = get_response_df(['MNM', 'malaria no more round b 1-shot'], 
                      db_conf, 
                      ['stratumid'], 
                      'stratumid')

ddb = ddb \
    .merge(treatment_assignment, how='right', left_on='stratumid', right_on='disthash') \
    .merge(dist_info, left_on='stratumid', right_on='clusterid')

ddb = ddb[ddb.wave == '0']

In [255]:
import numpy as np

def as_type(series, type_):
    return series.astype(type_)

def binarize(series, pos):
    s = as_type((series == pos), int)
    s[series.isna()] = np.nan
    return s

def binarize_cols(df, cols):
    df = df.copy()
    for col, target in cols:
        df[col] = binarize(df[col], target)
    return df

In [256]:
cols = [
    ('malaria2weeks', 'Yes'), 
    ('fever2weeks', 'Yes'), 
    ('seekhelpfever', 'Yes'),
    ('longsleeves', 'Yes'),
    ('sleepundernet', 'Yes'),
    ('timeseekhelpfever', 'Less than 24 hours')
]

dda = binarize_cols(dda, cols)

In [175]:
cols = [
    ('malaria4months', 'Yes'), 
    ('fever4months', 'Yes'), 
    ('seekhelpfever', 'Yes'),
    ('longsleeves', 'Yes'),
    ('sleepundernet', 'Yes'),
    ('timeseekhelpfever', 'Less than 24 hours')
]

ddb = binarize_cols(ddb, cols)

In [221]:
def aggs(df, sums, anys, means):
    r = df.iloc[0]
    # fil = df.wave > 1
    fil = df.week > 40 # 36 = first week of september
    for var in sums:
        r[var] = df[(fil)][var].sum()
    for var in anys:
        r[var] = int(df[(fil)][var].sum() > 0)
    for var in means:
        r[var] = df[(fil)][var].mean()

    r['wave'] = None
    r['waves_answered'] = df[(fil)].shape[0]
    return r


# feature matrix

# user, sum bednet sleeps in follows, if malaria at any point, binary controls (malaria in last 5 years?)

mat = binarize_cols(d, cols) \
    .sort_values('week') \
    .groupby('userid') \
    .apply(lambda df: aggs(df, 
                           sums=['seekhelpfever', 'timeseekhelpfever', 'fever2weeks'], 
                           anys=['malaria2weeks'], 
                           means=['sleepundernet', 'longsleeves', 'bednet_share'])) \
    .reset_index(drop=True) \
    .dropna(subset=['sleepundernet'])



In [136]:
mat['seekhelp_if_fever'] = mat.seekhelpfever / mat.fever2weeks

In [ ]:
ks = mat.groupby(['clusterid', 'treatment']).kutcha.mean()

In [ ]:
ks

In [69]:
def as_type(series, type_):
    return series.astype(type_)

def quantile(series, q):
    return series > series.quantile(q)

In [64]:
dd['unet'] = dd.under_net.astype(int)

In [145]:
dd['malaria_now_int'] = dd.malaria_now.astype(int)

In [ ]:
dd[dd.wave == 0].malaria_now

In [156]:
m = sm.BinomialBayesMixedGLM.from_formula('malaria_now_int ~ treatment', {'a': '0 + C(clusterid)'}, dd[dd.wave == 0])
mm = m.fit_vb()

In [ ]:
mm.random_effects()

In [ ]:
mm.summary()

In [ ]:
m = sm.BinomialBayesMixedGLM.from_formula('unet ~ treatment', {'a': '0 + C(userid)', 'b': '0 + C(clusterid)'}, dd)
mm = m.fit()

In [ ]:
mat[mat.under_net.isna()]

In [ ]:
md = smf.mixedlm("long_sleeves ~ treatment", mat, groups=mat.clusterid.astype('category').cat.codes)

In [ ]:
md.fit().summary()

In [ ]:
m.summary()

In [ ]:
mat[mat.seekhelp_if_fever.notna()].groupby(['treatment']).seekhelp_if_fever.value_counts(normalize=True)

In [189]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np


def get_percentages(base, treatment):
    return expit(base), expit(base + treatment), 1 - ((1 - expit(base + treatment))) / (1 - expit(base))


def reg(form, dat, weights_var='waves_answered'):
    try:
        model = smf.glm(form, dat, family=sm.families.Binomial(), var_weights=dat[weights_var])
    except KeyError:
        return smf.logit(form, dat) \
                   .fit(cov_type='cluster', cov_kwds={'groups': dat['stratumid']}) \
                   .summary()

In [ ]:
smf.ols('bednet_share ~ treatment', data=mat[mat.hasmosquitonet == 'Yes'], var_weights=mat[mat.hasmosquitonet == 'Yes']['waves_answered']).fit().summary()

In [ ]:
reg('bednet_share ~ treatment*pregnantwoman', mat[mat.hasmosquitonet == 'Yes'])

In [ ]:
reg('sleepundernet ~ treatment*pregnantwoman', mat[mat.hasmosquitonet == 'Yes'])

In [90]:
matb = ddb

matb['seekhelp_if_fever'] = matb['seekhelpfever']
matb['fever2weeks'] = (matb['fever4months'] == 1).astype(int)

In [94]:
f = ddb[ddb.fever4months.notna()]['stratumid'].astype('category').cat.codes.values

In [ ]:
ddb.columns

In [154]:
ddb.dwelling.unique()

array([nan, 'Semi-pucca', 'Kutcha (made of mud, tin, straw)',
       'Pucca (have cement/brick wall and floor'], dtype=object)

In [193]:
ddb['kutcha'] = ddb.dwelling == 'Kutcha (made of mud, tin, straw)'
ddb['pucca'] = ddb.dwelling == 'Pucca (have cement/brick wall and floor'

In [159]:
ddb.education.value_counts()

University degree or higher    2645
Secondary                      1335
Primary                         533
Never attended school           247
Name: education, dtype: int64

In [195]:
reg('ths ~ pucca', ddb[ddb.dwelling.notna() & ddb.fever4months.notna()])

Optimization terminated successfully.
         Current function value: 0.460576
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           fever4months   No. Observations:                 4912
Model:                          Logit   Df Residuals:                     4910
Method:                           MLE   Df Model:                            1
Date:                Tue, 23 Feb 2021   Pseudo R-squ.:               0.0002312
Time:                        10:33:39   Log-Likelihood:                -2262.4
converged:                       True   LL-Null:                       -2262.9
Covariance Type:              cluster   LLR p-value:                    0.3063
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.5983      0.055    -29.050      0.000      -1.706      -1.490
pucca[T.True]     0.0777      0.082      0.950      0.342      -0.083       0.238
=================================================================================
"""

In [203]:
reg('seekhelpfever ~ treatment*pucca', ddb[ddb.dwelling.notna() & ddb.fever4months.notna() & ddb.seekhelpfever.notna()])

Optimization terminated successfully.
         Current function value: 0.304920
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          seekhelpfever   No. Observations:                  852
Model:                          Logit   Df Residuals:                      848
Method:                           MLE   Df Model:                            3
Date:                Tue, 23 Feb 2021   Pseudo R-squ.:                 0.01254
Time:                        10:37:02   Log-Likelihood:                -259.79
converged:                       True   LL-Null:                       -263.09
Covariance Type:              cluster   LLR p-value:                   0.08582
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   1.8741      0.198      9.484      0.000       1.487       2.261
pucca[T.True]               0.5407      0.345      1.569      0.117      -0.135       1.216
treatment                   0.6108      0.347      1.758      0.079      -0.070       1.292
treatment:pucca[T.True]    -0.4257      0.538     -0.791      0.429      -1.480       0.629
===========================================================================================
"""

In [206]:
get_percentages(1.87, 0.6108)

(0.8664582774667876, 0.9227848195437104, 0.4217898422188925)

In [167]:
reg('seekhelpfever ~ treatment*dwelling', ddb[ddb.dwelling.notna() & ddb.fever4months.notna() & ddb.seekhelpfever.notna()])

Optimization terminated successfully.
         Current function value: 0.355515
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          seekhelpfever   No. Observations:                  677
Model:                          Logit   Df Residuals:                      671
Method:                           MLE   Df Model:                            5
Date:                Thu, 18 Feb 2021   Pseudo R-squ.:                 0.01323
Time:                        10:41:37   Log-Likelihood:                -240.68
converged:                       True   LL-Null:                       -243.91
Covariance Type:              cluster   LLR p-value:                    0.2647
=================================================================================================================================
                                                                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.4843      0.262      5.672      0.000       0.971       1.997
dwelling[T.Pucca (have cement/brick wall and floor]               0.6776      0.396      1.713      0.087      -0.098       1.453
dwelling[T.Semi-pucca]                                            0.3075      0.415      0.741      0.459      -0.506       1.121
treatment                                                         0.6492      0.525      1.236      0.217      -0.380       1.679
treatment:dwelling[T.Pucca (have cement/brick wall and floor]    -0.4561      0.642     -0.710      0.478      -1.715       0.802
treatment:dwelling[T.Semi-pucca]                                 -0.2029      0.727     -0.279      0.780      -1.629       1.223
=================================================================================================================================
"""

In [153]:
get_percentages(1.48, 0.6492)

(0.8145725807070178, 0.8937090377022275, 0.4267786139555295)

In [136]:
get_percentages(1.573, 0.2663)

(0.8282108615242946, 0.8628658986241335, 0.20173008263115488)

In [119]:
m = pd.concat([mat, matb])
reg('seekhelp_if_fever ~ treatment', m[m.seekhelp_if_fever.notna()], 'fever2weeks')

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:      seekhelp_if_fever   No. Observations:                 1598
Model:                            GLM   Df Residuals:                     1596
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -804.38
Date:                Sun, 07 Feb 2021   Deviance:                       1516.7
Time:                        18:02:54   Pearson chi2:                 1.76e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6309      0.087     18.687      0.000       1.460       1.802
treatment      0.1977      0.127      1.561      0.118      -0.050       0.446
==============================================================================
"""

In [137]:
get_percentages(1.6309, 0.1977)

(0.8362928924541598, 0.8615948625093831, 0.15455633194263352)

In [137]:
reg('seekhelp_if_fever ~ treatment', mat[mat.seekhelp_if_fever.notna()], 'fever2weeks')

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:      seekhelp_if_fever   No. Observations:                 1005
Model:                            GLM   Df Residuals:                     1003
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -556.27
Date:                Sun, 07 Feb 2021   Deviance:                       1020.5
Time:                        18:20:19   Pearson chi2:                 1.17e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6468      0.107     15.452      0.000       1.438       1.856
treatment      0.1449      0.151      0.960      0.337      -0.151       0.441
==============================================================================
"""

In [125]:
reg('fever4months ~ treatment', matb)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           fever4months   No. Observations:                 4951
Model:                            GLM   Df Residuals:                     4949
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1814.3
Date:                Sun, 07 Feb 2021   Deviance:                       3628.7
Time:                        18:12:45   Pearson chi2:                 4.95e+03
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9782      0.061    -32.593      0.000      -2.097      -1.859
treatment     -0.0339      0.088     -0.387      0.699      -0.206       0.138
==============================================================================
"""

In [126]:
reg('fever2weeks ~ treatment', mat)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            fever2weeks   No. Observations:                 6496
Model:                            GLM   Df Residuals:                     6494
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4306.4
Date:                Sun, 07 Feb 2021   Deviance:                       5633.2
Time:                        18:12:47   Pearson chi2:                 8.75e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.7039      0.041    -66.740      0.000      -2.783      -2.625
treatment      0.1145      0.056      2.041      0.041       0.005       0.224
==============================================================================
"""

In [115]:
reg('malaria4months ~ treatment', matb)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         malaria4months   No. Observations:                 5001
Model:                            GLM   Df Residuals:                     4999
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -965.49
Date:                Sun, 07 Feb 2021   Deviance:                       1931.0
Time:                        17:59:59   Pearson chi2:                 5.00e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.0466      0.095    -32.197      0.000      -3.232      -2.861
treatment      0.1270      0.132      0.962      0.336      -0.132       0.386
==============================================================================
"""

In [116]:
expit(-3.0466), expit(0.1270)

(0.045364488367139494, 0.5317073940719488)

In [130]:
reg('malaria2weeks ~ treatment', mat, 'foo')

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:          malaria2weeks   No. Observations:                 6496
Model:                            GLM   Df Residuals:                     6494
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1526.8
Date:                Sun, 07 Feb 2021   Deviance:                       3053.5
Time:                        18:15:45   Pearson chi2:                 6.50e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.6843      0.072    -37.365      0.000      -2.825      -2.543
treatment     -0.0320      0.102     -0.313      0.754      -0.232       0.168
==============================================================================
"""

In [132]:
expit(-2.6843), expit(-2.6943 - 0.0320)

(0.06390615823520344, 0.061439175217968434)

In [ ]:
dd[dd.wave == 0].groupby('treatment').long_sleeves.mean()

In [ ]:
reg('as_type(under_net, int) ~ treatment:as_type(wave, str) + avg_under_net', dd[(dd.userid.isin(mat[mat.net_ownership].userid)) & (dd.wave > 0)])

In [ ]:
reg('as_type(under_net, int) ~ treatment:as_type(month, str) + avg_under_net', dd[(dd.userid.isin(mat[mat.net_ownership].userid)) & (dd.wave > 0)])

In [ ]:
reg('as_type(long_sleeves, int) ~ treatment:as_type(month, str) + avg_long_sleeves', dd)

In [ ]:
reg('as_type(long_sleeves, int) ~ treatment', dd[dd.wave == 0])

In [ ]:
reg('as_type(under_net, int) ~ treatment', dd[dd.wave == 0])

In [ ]:
reg('as_type(malaria_now, int) ~ treatment', dd[dd.wave == 0])

In [ ]:
reg('malaria_now ~ treatment', mat)

In [ ]:
mat['num_under_net'].max()

In [ ]:
reg('num_under_net ~ treatment', mat[(mat['owns_net'] == 'Yes')]) 

In [ ]:
reg('long_sleeves ~ treatment + avg_long_sleeves', mat) 

In [ ]:
reg('under_net ~ treatment', mat[mat['owns_net'] == 'Yes'])

In [ ]:
reg('under_net ~ treatment*pregnant_in_house', mat[mat['owns_net'] == 'Yes'])

In [ ]:
reg('under_net ~ treatment*quantile(avg_malaria_now, 0.5)', mat[mat['owns_net'] == 'Yes'])

In [ ]:
reg('under_net ~ treatment*kutcha*quantile(avg_malaria_now, 0.5)', mat[mat['owns_net'] == 'Yes'])

In [1413]:
expit(0.5991)

0.6454503734274153

In [1412]:
expit(0.5991 + 0.023)

0.6506960099929763

In [1416]:
expit(0.5991 - 0.2387)

0.5891372593292695

In [1414]:
expit(0.5991 + 0.023 + 0.1558 - 0.2387)

0.6316262977304613

In [ ]:
reg('under_net ~ treatment', mat[mat['owns_net'] == 'Yes'])

In [1379]:
expit(0.529)

0.6292498476593299

In [1388]:
expit(0.48)

0.617747874769249

In [1389]:
expit(0.58)

0.6410674063348171

In [1405]:
expit(0.349), expit(0.611)

(0.5863750605361624, 0.648168881838421)

In [1404]:
expit(0.58 - 0.09), expit(0.58 + 0.289)

(0.6201064323430902, 0.7045375762042413)

0.7045375762042413

In [ ]:
reg('under_net ~ treatment*lower_caste*quantile(avg_malaria_now, 0.5) + avg_under_net', mat[mat['owns_net'] == 'Yes'])

In [ ]:
reg('long_sleeves ~ treatment*lower_caste*quantile(avg_malaria_now, 0.5) + avg_long_sleeves', mat)

In [415]:
mat.malaria_now.sum()

0

In [410]:
mat.groupby('clusterid').malaria_now.sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [383]:
mat.groupby('treatment').malaria_now.value_counts(normalize=True)

treatment  malaria_now
0          0              0.917864
           1              0.082136
1          0              0.910769
           1              0.089231
Name: malaria_now, dtype: float64

In [ ]:
df.clusterid

In [ ]:
df.wave == 0

In [768]:
dist_info['high_malaria_district'] = dist_info.avg_malaria_now > 0.02

In [495]:
dd[dd.wave == 0].groupby('clusterid').apply(lambda df: df.malaria_now.sum()).values

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# recruit
# give some money

# randomize within cluster -> some ask questions, some follow up later.

# for follow-up later group, make stratified assignment, send ads to half

# for other group, nothing. 


# wait 2 weeks
# (advertise)
# send follow up


# follow up should be able to: A) test direct effect (t/c within our study) AND test effect of large-scale...



# 

In [ ]:
dist_info

In [ ]:
mm

In [543]:
smf.logit('malaria_now ~ treatment*avg_malaria_now*cost_per_message', mm).fit().summary()

Optimization terminated successfully.
         Current function value: 0.287730
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            malaria_now   No. Observations:                 1949
Model:                          Logit   Df Residuals:                     1941
Method:                           MLE   Df Model:                            7
Date:                Wed, 02 Dec 2020   Pseudo R-squ.:                 0.01610
Time:                        16:37:02   Log-Likelihood:                -560.79
converged:                       True   LL-Null:                       -569.96
Covariance Type:            nonrobust   LLR p-value:                   0.01047
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                     -1.9847      0.738     -2.689      0.007      -3.431      -0.538
treatment                                      0.4164      1.093      0.381      0.703      -1.725       2.558
avg_malaria_now                                0.0445     38.359      0.001      0.999     -75.138      75.227
treatment:avg_malaria_now                    -11.6342     51.275     -0.227      0.821    -112.130      88.862
cost_per_message                              -0.0403      0.034     -1.195      0.232      -0.106       0.026
treatment:cost_per_message                    -0.0314      0.054     -0.581      0.561      -0.138       0.075
avg_malaria_now:cost_per_message               0.9171      1.603      0.572      0.567      -2.225       4.059
treatment:avg_malaria_now:cost_per_message     0.9822      2.285      0.430      0.667      -3.496       5.461
==============================================================================================================
"""

In [438]:
df[(df.variable == 'net_lastnight') & (df.wave == 0)].groupby('treatment').translated_response.value_counts(normalize=True)

treatment  translated_response
0          Yes                    0.569815
           No                     0.430185
1          Yes                    0.562628
           No                     0.437372
Name: translated_response, dtype: float64

In [440]:
df.timestamp

1197     2020-07-23 08:06:51.476000+00:00
1190     2020-07-23 08:07:00.147000+00:00
1165     2020-07-23 08:07:55.339000+00:00
1181     2020-07-23 08:08:15.220000+00:00
1196     2020-07-23 08:08:21.454000+00:00
                       ...               
119064   2020-12-02 08:32:00.924000+00:00
95367    2020-12-02 08:57:51.108000+00:00
95615    2020-12-02 08:58:27.031000+00:00
95616    2020-12-02 08:59:03.112000+00:00
95623    2020-12-02 08:59:11.999000+00:00
Name: timestamp, Length: 83171, dtype: datetime64[ns, psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)]

In [439]:
df[(df.variable == 'net_lastnight') & (df.wave > 5)].groupby('treatment').translated_response.value_counts(normalize=True)

treatment  translated_response
0          Yes                    0.519062
           No                     0.480938
1          Yes                    0.572917
           No                     0.427083
Name: translated_response, dtype: float64

In [198]:
df[(df.variable == 'net_lastnight') & (df.wave > 0)] \
    .groupby('userid') \
    .apply(lambda df: df.assign(summed = (df.translated_response == 'Yes').sum())) \
    .groupby('treatment') \
    .summed.mean()

treatment
0.0    3.153181
1.0    3.222683
Name: summed, dtype: float64

In [ ]:
df[(df.variable == 'long_sleeves') & (df.wave == 0)].groupby('treatment').translated_response.value_counts(normalize=True)

In [408]:
df[(df.variable == 'long_sleeves') & (df.wave > 5)].groupby('treatment').translated_response.value_counts(normalize=True)

treatment  translated_response
0          Yes                    0.675000
           No                     0.325000
1          Yes                    0.653153
           No                     0.346847
Name: translated_response, dtype: float64

In [96]:
foo = df[(df.variable == 'net_lastnight') & (df.wave == 0)].translated_response.unique()
wrong = [i for i in foo if i not in ['Yes', 'No']]

ddd = df[(df.variable == 'net_lastnight') & (df.wave == 0) & df.translated_response.isin(wrong)]

In [104]:
form_lookup['0c9776ba-7805-4949-b921-1797e37830ab']['shortcode']

'baselinehinexc'

In [105]:
form_lookup['d9de2649-b403-41c9-8db4-8d41b36342e5']['shortcode']

'baselinehinexc'

In [ ]:
ddd[['surveyid', 'shortcode', 'question_ref', 'tref']]

In [69]:
df.treatment.value_counts()

0.0    71763
1.0    71691
Name: treatment, dtype: int64

In [11]:
from adopt.responses import get_response_df
from datetime import datetime, timezone
from adopt.clustering import only_target_users
import pandas as pd
from adopt.clustering import add_res_cols
from adopt.forms import response_translator
from adopt.malaria import shape_df
import re


def filter_time(df, min_date, max_date):
    min_date = datetime(*min_date, tzinfo=timezone.utc)
    max_date = datetime(*max_date, tzinfo=timezone.utc)

    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]


def ref_translation(ref_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup

def get_filtered_responses(survey_user, eng_form, other_form, shortcodes, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, shortcodes, refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)

def malaria_prob(groupby, key='malaria'):
    d = groupby[key].value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d[key] == True].drop(columns=key)
    return d


def stats(df):
    targets = df.kutcha.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])


def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
           for qt, q in zip(a['fields'], b['fields'])
           if qt['ref'] in refs}
    
    return [(n, lookup[ref], ts[ref]) for n, ref in questions]


In [ ]:
forms[1]['field']

In [ ]:
forms[1]

In [ ]:
filtered_res

In [21]:
# filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['follow1hin'], questions, db_conf)
filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['baselinehin', 'baselinehinexc'], questions, db_conf)

col_names = col_translators(forms[0], forms[1], questions)
rr = add_res_cols(col_names, shape_df(filtered_res)) \
    .reset_index(drop=True) \
    .dropna(subset=[r for r, _ in questions if not re.match('if_', r)])

In [ ]:
# choices: 
#   ordered: true
#   alphabetize: true
#   booleanize: Yes

In [12]:
dd = rr.merge(districts, left_on='md:clusterid', right_on='disthash')
qrefs = [c for c in dd.columns if len(c) > 35]
dd = dd.drop(columns=qrefs)

dd['under_net'] = dd.net_lastnight == 'Yes'
# dd['malaria'] = dd.malaria_incidence == 'Yes'
dd['malaria_now'] = dd.malaria_now == 'Yes'
# dd['kutcha'] = dd.home == 'Kutcha (made of mud, tin, straw)'
# dd['pucca'] = dd.home == 'Pucca (have cement/brick wall and floor'
# dd['university'] = dd.education == 'University degree or higher'
# dd['unemployed'] = dd.occupation == 'Unemployed'
dd['disthash'] = dd['md:clusterid']

NameError: name 'rr' is not defined